In [1]:
%load_ext autoreload
%autoreload 2
from itertools import product
import warnings; warnings.simplefilter('ignore')

#from datasets import get_dataset
#from train_eval import cross_validation_with_val_set

#from gcn import GCN, GCNWithJK
#from graph_sage import GraphSAGE, GraphSAGEWithJK
#from gin import GIN0, GIN0WithJK, GIN, GINWithJK
#from graclus import Graclus
#from top_k import TopK
#from diff_pool import DiffPool
#from sag_pool import SAGPool
#from global_attention import GlobalAttentionNet
#from set2set import Set2SetNet
#from sort_pool import SortPool

import time, os
import argparse
import yaml
from pprint import pprint
import logging
import os

from pathlib import Path
import sys
import inspect
import pkgutil
from importlib import import_module

import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool, JumpingKnowledge
from torch import tensor
from torch.optim import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir+'/graph_classification') 

from TFM_graph_classification import *
from TFM_graph_classification_models import *
from TFM_function_renaming_dataset_creation import *

from TFM_function_renaming_dataset_creation import *
from TFM_function_renaming_dataset_creation import FunctionsDataset
from TFM_function_renaming_baseline_models import *
from TFM_function_renaming_preprocess_dataset_splits import *


import logging

logger = logging.getLogger('exp2')
#logger.basicConfig(filename='exp2-gnn.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')
f_handler = logging.FileHandler('exp2-gnn.log')
f_handler.setLevel(logging.DEBUG)
logger.addHandler(f_handler)

In [2]:

class GCN(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GCNConv(hidden, hidden))
        self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


class GCNWithJK(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes, mode='cat'):
        super(GCNWithJK, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GCNConv(hidden, hidden))
        self.jump = JumpingKnowledge(mode)
        if mode == 'cat':
            self.lin1 = Linear(num_layers * hidden, hidden)
        else:
            self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.jump.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        xs = [x]
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
            xs += [x]
        x = self.jump(xs)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__

In [3]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
from torch_geometric.nn import GINConv, global_mean_pool, JumpingKnowledge
from torch_geometric.nn import SAGEConv, GlobalAttention
from torch_geometric.nn import SAGEConv, global_mean_pool, JumpingKnowledge
from torch_geometric.nn import SAGEConv, Set2Set
from torch_geometric.nn import SAGEConv, global_sort_pool




class SortPool(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes):
        super(SortPool, self).__init__()
        self.k = 10
        self.conv1 = SAGEConv(dataset.num_features, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden, hidden))
        self.lin1 = Linear(self.k * hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


class Set2SetNet(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes):
        super(Set2SetNet, self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden, hidden))
        self.set2set = Set2Set(hidden, processing_steps=4)
        self.lin1 = Linear(2 * hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.set2set.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__




class GraphSAGE(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden, hidden))
        self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


class GraphSAGEWithJK(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes, mode='cat'):
        super(GraphSAGEWithJK, self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden, hidden))
        self.jump = JumpingKnowledge(mode)
        if mode == 'cat':
            self.lin1 = Linear(num_layers * hidden, hidden)
        else:
            self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.jump.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        xs = [x]
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
            xs += [x]
        x = self.jump(xs)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__
    
 
class GlobalAttentionNet(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes):
        super(GlobalAttentionNet, self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden, hidden))
        self.att = GlobalAttention(Linear(hidden, 1))
        self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.att.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


In [4]:


def cross_validation_with_val_set(dataset,model, folds, epochs, batch_size,
                                  lr, lr_decay_factor, lr_decay_step_size,
                                  weight_decay, logger=None):

    
    val_losses, accs, durations, f1macros = [], [], [], []
    val_f1macros  =[]
    for fold, (train_idx, test_idx,
               val_idx) in enumerate(zip(*k_fold(dataset, folds))):

        train_dataset = dataset[train_idx]
        test_dataset = dataset[test_idx]
        val_dataset = dataset[val_idx]

        if 'adj' in train_dataset[0]:
            train_loader = DenseLoader(train_dataset, batch_size, shuffle=True)
            val_loader = DenseLoader(val_dataset, batch_size, shuffle=False)
            test_loader = DenseLoader(test_dataset, batch_size, shuffle=False)
        else:
            train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

        model.to(device).reset_parameters()
        optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_start = time.perf_counter()

        
        try:
            for epoch in range(1, epochs + 1):
                train_loss = train(model, optimizer, train_loader)
                val_losses.append(eval_loss(model, val_loader))
                val_f1macros.append(eval_f1macro(model, val_loader))
                accs.append(eval_acc(model, test_loader))
                f1macros.append(eval_f1macro(model, test_loader))
                eval_info = {
                    'fold': fold,
                    'epoch': epoch,
                    'train_loss': train_loss,
                    'val_loss': val_losses[-1],
                    'val_f1macro': val_f1macros[-1],
                    'test_acc': accs[-1],
                    'test_f1-macro': f1macros[-1],
                    'classif_report': eval_classification_report(model, test_loader)
                }

                if logger is not None:
                    logger(eval_info)

                if epoch % lr_decay_step_size == 0:
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr_decay_factor * param_group['lr']

            if torch.cuda.is_available():
                torch.cuda.synchronize()

            t_end = time.perf_counter()
            durations.append(t_end - t_start)
        except:
            for epoch in range(1, epochs + 1):
                train_loss = 1e1000000
                val_losses.append(1e1000000)
                val_f1macros.append(0.0)
                accs.append(0.0)
                f1macros.append(0.0)
                eval_info = {
                    'fold': fold,
                    'epoch': epoch,
                    'train_loss': train_loss,
                    'val_loss': val_losses[-1],
                    'val_f1macro': val_f1macros[-1],
                    'test_acc': accs[-1],
                    'test_f1-macro': f1macros[-1],
                    'classif_report': eval_classification_report(model, test_loader)
                }

                if logger is not None:
                    logger(eval_info)

            t_end = time.perf_counter()
            durations.append(t_end - t_start)

    loss, acc, duration = tensor(val_losses), tensor(accs), tensor(durations)
    loss, acc = loss.view(folds, epochs), acc.view(folds, epochs)
    loss, argmin = loss.min(dim=1)
    val_f1macro = tensor(val_f1macros)
    val_f1macro = val_f1macro.view(folds, epochs)
    val_f1macro = val_f1macro[torch.arange(folds, dtype=torch.long), argmin]
    
    acc = acc[torch.arange(folds, dtype=torch.long), argmin]
    f1macro = tensor(f1macros)
    f1macro = f1macro.view(folds, epochs)
    f1macro = f1macro[torch.arange(folds, dtype=torch.long), argmin]

    loss_mean = loss.mean().item()
    val_f1macro_mean = val_f1macro.mean().item()
    acc_mean = acc.mean().item()
    acc_std = acc.std().item()
    f1macro_mean = f1macro.mean().item()
    f1macro_std = f1macro.std().item()
    duration_mean = duration.mean().item()
    print('Val Loss: {:.4f}, Val f1macro: {:.4f}, Test Accuracy: {:.3f} ± {:.3f},Test F1-macro: {:.3f} ± {:.3f}, Duration: {:.3f}'.
          format(loss_mean, val_f1macro_mean, acc_mean, acc_std, f1macro_mean, f1macro_std, duration_mean))

    return loss_mean, acc_mean, acc_std


def k_fold(dataset, folds):
    skf = StratifiedKFold(folds, shuffle=True, random_state=12345)

    test_indices, train_indices = [], []
    #for _, idx in skf.split(torch.zeros(len(dataset)), dataset.data.y):
    for _, idx in skf.split(torch.zeros(len(dataset)), dataset.y):
        test_indices.append(torch.from_numpy(idx))

    val_indices = [test_indices[i - 1] for i in range(folds)]

    for i in range(folds):
        train_mask = torch.ones(len(dataset), dtype=torch.bool)
        train_mask[test_indices[i]] = 0
        train_mask[val_indices[i]] = 0
        train_indices.append(train_mask.nonzero().view(-1))

    return train_indices, test_indices, val_indices


def num_graphs(data):
    if data.batch is not None:
        return data.num_graphs
    else:
        return data.x.size(0)


def train(model, optimizer, loader):
    model.train()

    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        data = data.to(device)
        out = model(data)
        loss = F.nll_loss(out, data.y.view(-1))
        loss.backward()
        total_loss += loss.item() * num_graphs(data)
        optimizer.step()
    return total_loss / len(loader.dataset)


def eval_acc(model, loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            pred = model(data).max(1)[1]
        correct += pred.eq(data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)

def eval_f1macro(model, loader, labels=list(range(10)) ):
    model.eval()

    #correct = 0
    y_true = []
    y_pred = []
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            pred = model(data).max(1)[1]
        y_pred.extend(pred.flatten().tolist())
        y_true.extend(data.y.flatten().tolist())
        #correct += pred.eq(data.y.view(-1)).sum().item()
    from pprint import pprint  
    
    #print(len(y_true),len(y_pred))
    #pprint(y_true)
    #pprint(y_pred)
    #pprint(labels)
    
    return f1_score(y_true, y_pred, average='macro', labels=labels)  
    #return correct / len(loader.dataset)

    
def eval_classification_report(model, loader):
    model.eval()

    #correct = 0
    y_true = []
    y_pred = []
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            pred = model(data).max(1)[1]
        y_pred.extend(pred.flatten().tolist())
        y_true.extend(data.y.flatten().tolist())
        #correct += pred.eq(data.y.view(-1)).sum().item()
        
    return classification_report(y_true, y_pred, output_dict=True)  
    #return correct / len(loader.dataset)


def eval_loss(model, loader):
    model.eval()

    loss = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
        loss += F.nll_loss(out, data.y.view(-1), reduction='sum').item()
    return loss / len(loader.dataset)

In [5]:
def logger(info):
    from pprint import pprint
    fold, epoch = info['fold'] + 1, info['epoch']
    val_loss, test_acc, test_f1macro, report = info['val_loss'], info['test_acc'], info['test_f1-macro'], info['classif_report']
    val_f1macro = info['val_f1macro']
    print('{:02d}/{:03d}: Val Loss: {:.4f}, Val F1macro: {:.4f},Test Accuracy: {:.3f}, Test F1-macro: {:.3f}'.format(
        fold, epoch, val_loss,val_f1macro, test_acc,test_f1macro))
    #pprint(report)


## Tfidf add to graph

In [7]:
from TFM_function_renaming_nlp_models import *
#add_tfidf_to_dataset('./tmp/symbols_dataset_3_precomp_split_undersample_max')
add_tfidf_to_dataset_with_a_trick('./tmp/symbols_dataset_3_precomp_split_undersample_max')

processed_path tmp/symbols_dataset_3_precomp_split_undersample_max/training_set/processed
processed_path tmp/symbols_dataset_3_precomp_split_undersample_max/test_set/processed


In [8]:
dataset = FunctionsDataset(root='./tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set')
#dataset.gnn_mode_off()
dir(dataset[0])

processed_path tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set/processed


['__call__',
 '__cat_dim__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__inc__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply',
 'clone',
 'coalesce',
 'code_feats',
 'contains_isolated_nodes',
 'contains_self_loops',
 'contiguous',
 'debug',
 'edge_attr',
 'edge_index',
 'filename',
 'from_dict',
 'is_coalesced',
 'is_directed',
 'is_undirected',
 'keys',
 'label',
 'num_edge_features',
 'num_edges',
 'num_faces',
 'num_features',
 'num_node_features',
 'num_nodes',
 'pos',
 'tfidf_vec',
 'to',
 'x',
 'x_topo_feats',
 'x_topo_times',
 'y']

In [9]:
dataset[0].edge_attr

tensor([0.1360, 0.0000, 0.0000, 0.0000, 0.0000, 0.0506, 0.0000, 0.0000, 0.0000,
        0.1012, 0.0000, 0.0000, 0.4687, 0.4687, 0.2654, 0.0630, 0.0713, 0.0000,
        0.0000, 0.4129, 0.0000, 0.0645, 0.0671, 0.0000, 0.0000, 0.1091, 0.0000,
        0.0000, 0.0000, 0.1338, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2912, 0.0000, 0.0000,
        0.0000, 0.3470, 0.0000, 0.1437, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0504, 0.0506, 0.0000,
        0.0544, 0.0830, 0.0000, 0.0000, 0.0000])

In [10]:
dataset[0].tfidf_vec

tensor([0.1360, 0.0000, 0.0000, 0.0000, 0.0000, 0.0506, 0.0000, 0.0000, 0.0000,
        0.1012, 0.0000, 0.0000, 0.4687, 0.4687, 0.2654, 0.0630, 0.0713, 0.0000,
        0.0000, 0.4129, 0.0000, 0.0645, 0.0671, 0.0000, 0.0000, 0.1091, 0.0000,
        0.0000, 0.0000, 0.1338, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2912, 0.0000, 0.0000,
        0.0000, 0.3470, 0.0000, 0.1437, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0504, 0.0506, 0.0000,
        0.0544, 0.0830, 0.0000, 0.0000, 0.0000])

In [15]:
from TFM_function_renaming_nlp_models import *
add_tfidf_to_dataset('./tmp/symbols_dataset_1_precomp_split_undersample_max')
#add_tfidf_to_dataset_with_a_trick('./tmp/symbols_dataset_1_precomp_split_undersample_max')

processed_path tmp/symbols_dataset_1_precomp_split_undersample_max/training_set/processed
processed_path tmp/symbols_dataset_1_precomp_split_undersample_max/test_set/processed


In [16]:
dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
#dataset.gnn_mode_off()
dir(dataset[0])

processed_path tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set/processed


['__call__',
 '__cat_dim__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__inc__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply',
 'clone',
 'coalesce',
 'code_feats',
 'contains_isolated_nodes',
 'contains_self_loops',
 'contiguous',
 'debug',
 'edge_attr',
 'edge_index',
 'filename',
 'from_dict',
 'is_coalesced',
 'is_directed',
 'is_undirected',
 'keys',
 'label',
 'num_edge_features',
 'num_edges',
 'num_faces',
 'num_features',
 'num_node_features',
 'num_nodes',
 'pos',
 'tfidf_vec',
 'to',
 'x',
 'x_topo_feats',
 'x_topo_times',
 'y']

In [17]:
dataset[0].edge_attr

tensor([0.0702, 0.0882, 0.0816, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0520, 0.0622, 0.0000, 0.0000, 0.0000, 0.4552, 0.2024, 0.0755, 0.2288,
        0.0000, 0.3481, 0.0000, 0.1347, 0.1402, 0.0568, 0.0579, 0.1128, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0598, 0.0691,
        0.0000, 0.0000, 0.0000, 0.2008, 0.0000, 0.0000, 0.1506, 0.0705, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1487, 0.0797, 0.0628, 0.0560, 0.0000, 0.0000,
        0.0000, 0.0000, 0.1593, 0.0000, 0.0000, 0.0000, 0.1522, 0.0665, 0.0000,
        0.1885, 0.0778, 0.0814, 0.0559, 0.0767, 0.0000, 0.0658, 0.3634, 0.0472,
        0.0000, 0.0000, 0.0000, 0.0479, 0.2121, 0.0000, 0.0000, 0.1715, 0.1752,
        0.0000, 0.1245, 0.0772])

In [18]:
dataset[0].tfidf_vec

tensor([0.0702, 0.0882, 0.0816, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0520, 0.0622, 0.0000, 0.0000, 0.0000, 0.4552, 0.2024, 0.0755, 0.2288,
        0.0000, 0.3481, 0.0000, 0.1347, 0.1402, 0.0568, 0.0579, 0.1128, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0598, 0.0691,
        0.0000, 0.0000, 0.0000, 0.2008, 0.0000, 0.0000, 0.1506, 0.0705, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1487, 0.0797, 0.0628, 0.0560, 0.0000, 0.0000,
        0.0000, 0.0000, 0.1593, 0.0000, 0.0000, 0.0000, 0.1522, 0.0665, 0.0000,
        0.1885, 0.0778, 0.0814, 0.0559, 0.0767, 0.0000, 0.0658, 0.3634, 0.0472,
        0.0000, 0.0000, 0.0000, 0.0479, 0.2121, 0.0000, 0.0000, 0.1715, 0.1752,
        0.0000, 0.1245, 0.0772])

### Results:
The precomputed tfidf_vec is now present on the the tfidf_vec and edge_attr attributes of each graph in the dataset. We can now proceed to modify the GNN models to take this into account and concat the GNN embedding with the tfidf embedding(from the edge_attr) to feed the last fc layers of the model.

## Modifying GNN models for precomputed tfidf bow

In [17]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
from torch_geometric.nn import GINConv, global_mean_pool, JumpingKnowledge
from torch_geometric.nn import SAGEConv, GlobalAttention
from torch_geometric.nn import SAGEConv, global_mean_pool, JumpingKnowledge
from torch_geometric.nn import SAGEConv, Set2Set
from torch_geometric.nn import SAGEConv, global_sort_pool




    
class GIN0(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes, bow_dim=84,):
        super(GIN0, self).__init__()
        self.conv1 = GINConv(Sequential(
            Linear(dataset.num_features, hidden),
            ReLU(),
            Linear(hidden, hidden),
            ReLU(),
            BN(hidden),
        ),
                             train_eps=False)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(
                GINConv(Sequential(
                    Linear(hidden, hidden),
                    ReLU(),
                    Linear(hidden, hidden),
                    ReLU(),
                    BN(hidden),
                ),
                        train_eps=False))
        self.lin1 = Linear(hidden+bow_dim, hidden)
        self.lin2 = Linear(hidden, num_classes)
        self.bow_dim=bow_dim

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch, bow_embedding = data.x, data.edge_index, data.batch, data.edge_attr
        x = self.conv1(x, edge_index)
        for conv in self.convs:
            x = conv(x, edge_index)
        x = global_mean_pool(x, batch)
        
        # combine gnn embedding and bow embedding
        x = torch.cat((x,bow_embedding.view(-1,self.bow_dim)),1)
        
        # fc layers
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


class GIN0WithJK(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes, mode='cat',bow_dim=84):
        super(GIN0WithJK, self).__init__()
        self.conv1 = GINConv(Sequential(
            Linear(dataset.num_features, hidden),
            ReLU(),
            Linear(hidden, hidden),
            ReLU(),
            BN(hidden),
        ),
                             train_eps=False)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(
                GINConv(Sequential(
                    Linear(hidden, hidden),
                    ReLU(),
                    Linear(hidden, hidden),
                    ReLU(),
                    BN(hidden),
                ),
                        train_eps=False))
        self.jump = JumpingKnowledge(mode)
        if mode == 'cat':
            self.lin1 = Linear(num_layers * hidden+bow_dim, hidden)
        else:
            self.lin1 = Linear(hidden+bow_dim, hidden)
        self.lin2 = Linear(hidden, num_classes)
        self.bow_dim=bow_dim

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.jump.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch, bow_embedding = data.x, data.edge_index, data.batch, data.edge_attr
        x = self.conv1(x, edge_index)
        xs = [x]
        for conv in self.convs:
            x = conv(x, edge_index)
            xs += [x]
        x = self.jump(xs)
        x = global_mean_pool(x, batch)
                         
        # combine gnn embedding and bow embedding
        #print("bow.shape", bow_embedding.shape)
        x = torch.cat((x,bow_embedding.view(-1,self.bow_dim)),1)
        #print("after x",x.shape)
        # fc layers               
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


class GIN(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes, bow_dim = 84):
        super(GIN, self).__init__()
        self.conv1 = GINConv(Sequential(
            Linear(dataset.num_features, hidden),
            ReLU(),
            Linear(hidden, hidden),
            ReLU(),
            BN(hidden),
        ),
                             train_eps=True)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(
                GINConv(Sequential(
                    Linear(hidden, hidden),
                    ReLU(),
                    Linear(hidden, hidden),
                    ReLU(),
                    BN(hidden),
                ),
                        train_eps=True))
        self.lin1 = Linear(hidden+bow_dim, hidden)
        self.lin2 = Linear(hidden, num_classes)
        self.bow_dim=bow_dim

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch, bow_embedding = data.x, data.edge_index, data.batch, data.edge_attr
        
        x = self.conv1(x, edge_index)
        for conv in self.convs:
            x = conv(x, edge_index)
        x = global_mean_pool(x, batch)
        
        # combine gnn embedding and bow embedding
        #print("x.shape", x.shape)
        #print("bow_embedding.shape",bow_embedding.shape)
        # there are 84 dimensions in the bow embedding
        x = torch.cat((x,bow_embedding.view(-1,self.bow_dim)),1)
        
        # fc layers
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


class GINWithJK(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes, mode='cat',bow_dim=84):
        super(GINWithJK, self).__init__()
        self.conv1 = GINConv(Sequential(
            Linear(dataset.num_features, hidden),
            ReLU(),
            Linear(hidden, hidden),
            ReLU(),
            BN(hidden),
        ),
                             train_eps=True)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(
                GINConv(Sequential(
                    Linear(hidden, hidden),
                    ReLU(),
                    Linear(hidden, hidden),
                    ReLU(),
                    BN(hidden),
                ),
                        train_eps=True))
        self.jump = JumpingKnowledge(mode)
        if mode == 'cat':
            self.lin1 = Linear(num_layers * hidden+bow_dim, hidden)
        else:
            self.lin1 = Linear(hidden+bow_dim, hidden)
        self.lin2 = Linear(hidden, num_classes)
        
        self.bow_dim=bow_dim

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.jump.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):         
        x, edge_index, batch, bow_embedding = data.x, data.edge_index, data.batch, data.edge_attr
        x = self.conv1(x, edge_index)
        xs = [x]
        for conv in self.convs:
            x = conv(x, edge_index)
            xs += [x]
        x = self.jump(xs)
        x = global_mean_pool(x, batch)
                         
        # combine gnn embedding and bow embedding
        x = torch.cat((x,bow_embedding.view(-1,self.bow_dim)),1)
        
        # fc layers
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__
    


## Deep HP-search on v1

In [34]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
results = []

batch_size=128
lr=0.01
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 4,]
hiddens = [16, ]  # the first fc layer will have hiddens+84.. 
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8

for Net in nets:
    for epochs in [5]:
        for num_layers, hidden in product(layers, hiddens):
            print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
            # rewrite here & definition to pass num_classes and num_features manually
            model = Net(dataset, num_layers, hidden, num_classes)

            # verify how validation set is obtained
            loss, acc, std = cross_validation_with_val_set(
                dataset,
                model,
                folds=10,
                epochs=epochs,
                batch_size=batch_size,
                lr=lr,
                lr_decay_factor=lr_decay_factor,
                lr_decay_step_size=lr_decay_step_size,
                weight_decay=0,
                logger=logger)

            if loss < best_result[0]:
                best_result = (loss, acc, std)

        desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
        print('Best result - {}'.format(desc))
        results += ['{} - {}: {}'.format(dataset_name, model, desc)]
        print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

processed_path tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set/processed
 model: GIN  epochs: 5  num_layers: 4  hidden: 16
num_features, data.x.shape
torch.Size([3, 1])
01/001: Val Loss: 2.0627, Val F1macro: 0.0928,Test Accuracy: 0.158, Test F1-macro: 0.097
01/002: Val Loss: 2.0426, Val F1macro: 0.0766,Test Accuracy: 0.148, Test F1-macro: 0.077
01/003: Val Loss: 2.0543, Val F1macro: 0.0777,Test Accuracy: 0.164, Test F1-macro: 0.081
01/004: Val Loss: 2.0375, Val F1macro: 0.0772,Test Accuracy: 0.176, Test F1-macro: 0.085
01/005: Val Loss: 2.0704, Val F1macro: 0.0552,Test Accuracy: 0.158, Test F1-macro: 0.069
02/001: Val Loss: 2.0824, Val F1macro: 0.0687,Test Accuracy: 0.148, Test F1-macro: 0.077
02/002: Val Loss: 2.0478, Val F1macro: 0.0884,Test Accuracy: 0.160, Test F1-macro: 0.078
02/003: Val Loss: 2.0470, Val F1macro: 0.1011,Test Accuracy: 0.158, Test F1-macro: 0.094
02/004: Val Loss: 2.0410, Val F1macro: 0.1012,Test Accuracy: 0.172, Test F1-macro: 0.109
02/005: Val 

In [ ]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=128
lr=0.005
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2, 4, 6]
hiddens = [32, 64, 128, 256]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8



for Net in nets:
    for epochs in [50,100]:
        for num_layers, hidden in product(layers, hiddens):
            print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
            # rewrite here & definition to pass num_classes and num_features manually
            model = Net(dataset, num_layers, hidden, num_classes)

            # verify how validation set is obtained
            loss, acc, std = cross_validation_with_val_set(
                dataset,
                model,
                folds=10,
                epochs=epochs,
                batch_size=batch_size,
                lr=lr,
                lr_decay_factor=lr_decay_factor,
                lr_decay_step_size=lr_decay_step_size,
                weight_decay=5e-4,
                logger=logger)

            if loss < best_result[0]:
                best_result = (loss, acc, std)

        desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
        print('Best result - {}'.format(desc))
        results += ['{} - {}: {}'.format(dataset_name, model, desc)]
        print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

In [ ]:
#V1 training  - concentrate on a algorithm and do a lot of epochs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=56
lr=0.005
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2]
hiddens = [ 64]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8



for Net in nets:
    for epochs in [500,1000]:
        for num_layers, hidden in product(layers, hiddens):
            print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
            # rewrite here & definition to pass num_classes and num_features manually
            model = Net(dataset, num_layers, hidden, num_classes)

            # verify how validation set is obtained
            loss, acc, std = cross_validation_with_val_set(
                dataset,
                model,
                folds=10,
                epochs=epochs,
                batch_size=batch_size,
                lr=lr,
                lr_decay_factor=lr_decay_factor,
                lr_decay_step_size=lr_decay_step_size,
                weight_decay=5e-4,
                logger=logger)

            if loss < best_result[0]:
                best_result = (loss, acc, std)

        desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
        print('Best result - {}'.format(desc))
        results += ['{} - {}: {}'.format(dataset_name, model, desc)]
        print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

In [ ]:
#V1 training  - concentrate on a algorithm and do a lot of epochs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=56
lr=0.005
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2]
hiddens = [ 64]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8



for Net in nets:
    for epochs in [2000]:
        for num_layers, hidden in product(layers, hiddens):
            print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
            # rewrite here & definition to pass num_classes and num_features manually
            model = Net(dataset, num_layers, hidden, num_classes)

            # verify how validation set is obtained
            loss, acc, std = cross_validation_with_val_set(
                dataset,
                model,
                folds=10,
                epochs=epochs,
                batch_size=batch_size,
                lr=lr,
                lr_decay_factor=lr_decay_factor,
                lr_decay_step_size=lr_decay_step_size,
                weight_decay=5e-4,
                logger=logger)

            if loss < best_result[0]:
                best_result = (loss, acc, std)

        desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
        print('Best result - {}'.format(desc))
        results += ['{} - {}: {}'.format(dataset_name, model, desc)]
        print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

In [ ]:
#V1 training  - concentrate on a algorithm and do a lot of epochs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=56
lr=0.005
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2]
hiddens = [ 64]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8



for Net in nets:
    for epochs in [2000]:
        for num_layers, hidden in product(layers, hiddens):
            print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
            # rewrite here & definition to pass num_classes and num_features manually
            model = Net(dataset, num_layers, hidden, num_classes)

            # verify how validation set is obtained
            loss, acc, std = cross_validation_with_val_set(
                dataset,
                model,
                folds=10,
                epochs=epochs,
                batch_size=batch_size,
                lr=lr,
                lr_decay_factor=lr_decay_factor,
                lr_decay_step_size=lr_decay_step_size,
                weight_decay=5e-4,
                logger=logger)

            if loss < best_result[0]:
                best_result = (loss, acc, std)

        desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
        print('Best result - {}'.format(desc))
        results += ['{} - {}: {}'.format(dataset_name, model, desc)]
        print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

## Now use a weight-decay on v1

In [ ]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
results = []

batch_size=128
lr=0.01
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2,4,8]
hiddens = [16,32,64,128,256 ]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8

for Net in nets:
    for epochs in [32]:
        for lr in [0.005]:
            for batch_size in [128]:
                for wd in [ 5e-4]:
                    for num_layers, hidden in product(layers, hiddens):
                        print("\n\n model:", Net.__name__,
                              " epochs:",epochs,
                              " num_layers:",num_layers,
                              " hidden:",hidden,
                              " batch_size:", batch_size,
                              " lr:",lr,
                              " wd:", wd
                             )
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
    print('Best result - {}'.format(desc))
    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

### Results:
* layers_hidden : 2-64, 4-64
* wd : 5e-3, 5e-4   <- pick one

In [ ]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
results = []

batch_size=128
lr=0.01
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2,4,8]
hiddens = [ 64 ]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8

for Net in nets:
    for epochs in [32]:
        for lr in [0.001]:
            for batch_size in [128]:
                for wd in [ 5e-4]:
                    for num_layers, hidden in [(8,128),(4,128),(4,256),(2,128),(2,64),(4,94)]:
                        print("\n\n model:", Net.__name__,
                              " epochs:",epochs,
                              " num_layers:",num_layers,
                              " hidden:",hidden,
                              " batch_size:", batch_size,
                              " lr:",lr,
                              " wd:", wd
                             )
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
    print('Best result - {}'.format(desc))
    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

### Results:
* wd : 5e-4  
* layers_hidden : 2-64, 4-256, 8-128 < 4-94  < 2-128 < 4-128
* batch_size: ?
* lr: ?
* epochs: ?
* model: ?

In [ ]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
results = []

batch_size=128
lr=0.01
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 4]
hiddens = [ 128 ]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8

for Net in nets:
    for epochs in [32]:
        for lr in [0.001,0.005,0.01]:
            for batch_size in [128,65,32]:
                for wd in [ 5e-4]:
                    for num_layers, hidden in [(4,128)]:
                        print("\n\n model:", Net.__name__,
                              " epochs:",epochs,
                              " num_layers:",num_layers,
                              " hidden:",hidden,
                              " batch_size:", batch_size,
                              " lr:",lr,
                              " wd:", wd
                             )
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
    print('Best result - {}'.format(desc))
    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

### Results:
* wd : 5e-4  
* layers_hidden : 2-64, 4-256, 8-128 < 4-94  < 2-128 < 4-128  <- pick top 3 or top 1 and 2 others
* batch_size: 128
* lr:  0.01 < 0.001 < 0.005
* epochs: ?
* model: ?

In [ ]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
results = []

batch_size=128
lr=0.005
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 4]
hiddens = [ 128 ]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    GINWithJK,    
    ##GCN,    
    GraphSAGE,    
    GIN0,    
    GIN,    
    GlobalAttentionNet,    
    Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8

for Net in nets:
    for epochs in [50]:
        for lr in [0.005]:
            for batch_size in [128]:
                for wd in [ 5e-4]:
                    for num_layers, hidden in [(4,128)]:
                        print("\n\n model:", Net.__name__,
                              " epochs:",epochs,
                              " num_layers:",num_layers,
                              " hidden:",hidden,
                              " batch_size:", batch_size,
                              " lr:",lr,
                              " wd:", wd
                             )
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
    print('Best result - {}'.format(desc))
    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

In [ ]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
results = []

batch_size=128
lr=0.005
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 4]
hiddens = [ 128 ]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    GIN0WithJK,    
    GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8

for Net in nets:
    for epochs in [50]:
        for lr in [0.005]:
            for batch_size in [128]:
                for wd in [ 5e-4]:
                    for num_layers, hidden in [(4,128)]:
                        print("\n\n model:", Net.__name__,
                              " epochs:",epochs,
                              " num_layers:",num_layers,
                              " hidden:",hidden,
                              " batch_size:", batch_size,
                              " lr:",lr,
                              " wd:", wd
                             )
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
    print('Best result - {}'.format(desc))
    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

### Results:
* wd : 5e-4  
* layers_hidden : 2-64, 4-256, 8-128 < 4-94  < 2-128 < 4-128  <- pick top 3 or top 1 and 2 others
* batch_size: 128
* lr:  0.005
* model: GIn < GIN0 < GINWJK
* epochs: ?


In [51]:
#V1 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
results = []

batch_size=128
lr=0.005
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 4]
hiddens = [ 128 ]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    #GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=8

for Net in nets:
    for epochs in [100,250,500,750,1000]:
        for lr in [0.005]:
            for batch_size in [128]:
                for wd in [ 5e-4]:
                    for num_layers, hidden in [(4,128)]:
                        print("\n\n model:", Net.__name__,
                              " epochs:",epochs,
                              " num_layers:",num_layers,
                              " hidden:",hidden,
                              " batch_size:", batch_size,
                              " lr:",lr,
                              " wd:", wd
                             )
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
    print('Best result - {}'.format(desc))
    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

processed_path tmp/symbols_dataset_1_precomp_split_undersample_max3/training_set/processed


 model: GINWithJK  epochs: 100  num_layers: 4  hidden: 128  batch_size: 128  lr: 0.005  wd: 0.0005
num_features, data.x.shape
torch.Size([3, 1])
Val Loss: 2.0105, Val f1macro: 0.1516, Test Accuracy: 0.204 ± 0.016,Test F1-macro: 0.143 ± 0.014, Duration: 439.259


 model: GINWithJK  epochs: 250  num_layers: 4  hidden: 128  batch_size: 128  lr: 0.005  wd: 0.0005
num_features, data.x.shape
torch.Size([3, 1])
Val Loss: 2.0116, Val f1macro: 0.1449, Test Accuracy: 0.201 ± 0.007,Test F1-macro: 0.138 ± 0.010, Duration: 1100.005


 model: GINWithJK  epochs: 500  num_layers: 4  hidden: 128  batch_size: 128  lr: 0.005  wd: 0.0005
num_features, data.x.shape
torch.Size([3, 1])
Val Loss: 2.0107, Val f1macro: 0.1473, Test Accuracy: 0.195 ± 0.013,Test F1-macro: 0.131 ± 0.008, Duration: 2172.227


 model: GINWithJK  epochs: 750  num_layers: 4  hidden: 128  batch_size: 128  lr: 0.005  wd: 0.0005
num_features, dat

## Hp-search on v3 dataset

In [9]:
#V3 training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=128
lr=0.001
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2, 4]
hiddens = [16, 32, 64, 128, 256]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    ##GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=19



for Net in nets:
    for epochs in [50]:
        for num_layers, hidden in product(layers, hiddens):
            print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
            # rewrite here & definition to pass num_classes and num_features manually
            model = Net(dataset, num_layers, hidden, num_classes, bow_dim=86)

            # verify how validation set is obtained
            loss, acc, std = cross_validation_with_val_set(
                dataset,
                model,
                folds=3,
                epochs=epochs,
                batch_size=batch_size,
                lr=lr,
                lr_decay_factor=lr_decay_factor,
                lr_decay_step_size=lr_decay_step_size,
                weight_decay=0.0005,
                logger=None)

            if loss < best_result[0]:
                best_result = (loss, acc, std)

        desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
        print('Best result - {}'.format(desc))
        results += ['{} - {}: {}'.format(dataset_name, model, desc)]
        print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

processed_path tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set/processed
 model: GIN  epochs: 50  num_layers: 2  hidden: 16
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6807, Val f1macro: 0.1108, Test Accuracy: 0.148 ± 0.005,Test F1-macro: 0.108 ± 0.018, Duration: 332.950
 model: GIN  epochs: 50  num_layers: 2  hidden: 32
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6613, Val f1macro: 0.1412, Test Accuracy: 0.159 ± 0.010,Test F1-macro: 0.145 ± 0.033, Duration: 364.269
 model: GIN  epochs: 50  num_layers: 2  hidden: 64
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6431, Val f1macro: 0.1779, Test Accuracy: 0.171 ± 0.009,Test F1-macro: 0.178 ± 0.019, Duration: 415.004
 model: GIN  epochs: 50  num_layers: 2  hidden: 128
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6260, Val f1macro: 0.1882, Test Accuracy: 0.181 ± 0.004,Test F1-macro: 0.192 ± 0.011, Duration: 418.960
 model: GIN  epochs: 50  num_layers: 2  hidden: 256
num_

In [18]:
#V3 training
# previous winner is 2layer 64 hidden units
# now testing 2l 64h + GCN/GraphSAGE/GIN + 30,35,40 epochs

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=128
lr=0.001
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2, ]
hiddens = [128]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    GIN0WithJK,    
    GINWithJK,    
    #GCN,    
    #GraphSAGE,    
    GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=19



for Net in nets:
    for epochs in [35]:
        for num_layers, hidden in [(2,128),(2,256)]:
            print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
            # rewrite here & definition to pass num_classes and num_features manually
            model = Net(dataset, num_layers, hidden, num_classes, bow_dim=86)

            # verify how validation set is obtained
            loss, acc, std = cross_validation_with_val_set(
                dataset,
                model,
                folds=3,
                epochs=epochs,
                batch_size=batch_size,
                lr=lr,
                lr_decay_factor=lr_decay_factor,
                lr_decay_step_size=lr_decay_step_size,
                weight_decay=0.0005,
                logger=None)

            if loss < best_result[0]:
                best_result = (loss, acc, std)

        desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
        print('Best result - {}'.format(desc))
        results += ['{} - {}: {}'.format(dataset_name, model, desc)]
        print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

processed_path tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set/processed
 model: GIN0WithJK  epochs: 35  num_layers: 2  hidden: 128
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6380, Val f1macro: 0.1718, Test Accuracy: 0.165 ± 0.009,Test F1-macro: 0.175 ± 0.010, Duration: 260.591
 model: GIN0WithJK  epochs: 35  num_layers: 2  hidden: 256
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6457, Val f1macro: 0.1615, Test Accuracy: 0.167 ± 0.007,Test F1-macro: 0.167 ± 0.016, Duration: 306.526
Best result - 0.165 ± 0.009
-----
assemblerv1 - GIN0WithJK: 0.165 ± 0.009
 model: GINWithJK  epochs: 35  num_layers: 2  hidden: 128
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6422, Val f1macro: 0.1719, Test Accuracy: 0.172 ± 0.009,Test F1-macro: 0.175 ± 0.007, Duration: 265.397
 model: GINWithJK  epochs: 35  num_layers: 2  hidden: 256
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6443, Val f1macro: 0.1662, Test Accuracy: 0.162 ± 0.009,T

IndexError: ('error getting: ', 3235)

In [14]:
342-86

256

In [ ]:
#V3 training
# previous winner is 2layer 64 hidden units
# now testing 2l 64h + GCN/GraphSAGE/GIN + 30,35,40 epochs

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=[128,64,32]
lr=[0.001,0.005,0.01]
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2, ]
hiddens = [128]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    GINWithJK,    
    #GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=19



for Net in nets:
    for lr in [0.001,0.005]:
        for batch_size in [128,64,32]:
            for wd in [5e-4,]:
                for epochs in [32]:
                    for num_layers, hidden in product(layers, hiddens):
                        print("\n\n model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes, bow_dim=86)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

                    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
                    print('Best result - {}'.format(desc))
                    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
                    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))

# best ones: 1, 2, 3, 5, 6, 10, 13
#  wd 5e-5, 5e-4, 5e-3
#  batch_size 128,64,32
#  lr 0.001

#10: wd 5-4, batch 32, lr 0.001
#13: wd 5e-5, batch 128, lr 0.005

In [ ]:
#V3 training
# previous winner is 2layer 64 hidden units
# now testing 2l 64h + GCN/GraphSAGE/GIN + 30,35,40 epochs

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

#batch_size=[128,64,32]
#lr=[0.001,0.005,0.01]
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2, ]
hiddens = [128]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    GINWithJK,    
    #GCN,    
    #GraphSAGE,    
    #GIN0,    
    #GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=19



for Net in nets:
    for lr in [0.001]:
        for batch_size in [32]:
            for wd in [ 5e-4]:
                for epochs in [30, 35, 40, 45, 50, 55, 60, 70, 80, 100, 200]:
                    for num_layers, hidden in product(layers, hiddens):
                        print(" model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes, bow_dim=86)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=None)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

                    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
                    print('Best result - {}'.format(desc))
                    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
                    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))




processed_path tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set/processed
 model: GINWithJK  epochs: 30  num_layers: 2  hidden: 128
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6680, Val f1macro: 0.1492, Test Accuracy: 0.157 ± 0.002,Test F1-macro: 0.150 ± 0.023, Duration: 253.006
Best result - 0.157 ± 0.002
-----
assemblerv1 - GINWithJK: 0.157 ± 0.002
 model: GINWithJK  epochs: 35  num_layers: 2  hidden: 128
num_features, data.x.shape
torch.Size([7, 4])
Val Loss: 2.6609, Val f1macro: 0.1580, Test Accuracy: 0.156 ± 0.009,Test F1-macro: 0.157 ± 0.017, Duration: 288.700
Best result - 0.156 ± 0.009
-----
assemblerv1 - GINWithJK: 0.157 ± 0.002
assemblerv1 - GINWithJK: 0.156 ± 0.009
 model: GINWithJK  epochs: 40  num_layers: 2  hidden: 128
num_features, data.x.shape
torch.Size([7, 4])


In [49]:
#V3 training
# previous winner is 2layer 64 hidden units
# now testing 2l 64h + GCN/GraphSAGE/GIN + 30,35,40 epochs

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
results = []

batch_size=[128,64,32]
lr=[0.001,0.005,0.01]
lr_decay_factor=0.5
lr_decay_step_size=50
layers = [ 2, ]
hiddens = [64]
nets = [   
    ##GCNWithJK, 
    ##GraphSAGEWithJK,    
    #GIN0WithJK,    
    #GINWithJK,    
    #GCN,    
    #GraphSAGE,    
    #GIN0,    
    GIN,    
    #GlobalAttentionNet,    
    #Set2SetNet, 
    
]

best_result = (float('inf'), 0, 0)  # (loss, acc, std)

dataset = FunctionsDataset(root='./tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set')
dataset_name='assemblerv1'
dataset.gnn_mode_on()
num_classes=19



for Net in nets:
    for lr in [0.001]:
        for batch_size in [32]:
            for wd in [ 5e-4]:
                for epochs in [500]:
                    for num_layers, hidden in product(layers, hiddens):
                        print("\n\n model:", Net.__name__," epochs:",epochs," num_layers:",num_layers," hidden:",hidden)
                        # rewrite here & definition to pass num_classes and num_features manually
                        model = Net(dataset, num_layers, hidden, num_classes, bow_dim=86)

                        # verify how validation set is obtained
                        loss, acc, std = cross_validation_with_val_set(
                            dataset,
                            model,
                            folds=3,
                            epochs=epochs,
                            batch_size=batch_size,
                            lr=lr,
                            lr_decay_factor=lr_decay_factor,
                            lr_decay_step_size=lr_decay_step_size,
                            weight_decay=wd,
                            logger=logger)

                        if loss < best_result[0]:
                            best_result = (loss, acc, std)

                    desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
                    print('Best result - {}'.format(desc))
                    results += ['{} - {}: {}'.format(dataset_name, model, desc)]
                    print('-----\n{}'.format('\n'.join(results)))


desc = '{:.3f} ± {:.3f}'.format(best_result[1], best_result[2])
print('Best result - {}'.format(desc))
results += ['{} - {}: {}'.format(dataset_name, model, desc)]
print('-----\n{}'.format('\n'.join(results)))




processed_path tmp/symbols_dataset_3_precomp_split_undersample_max2/training_set/processed


 model: GIN  epochs: 500  num_layers: 2  hidden: 64
num_features, data.x.shape
torch.Size([7, 4])
01/001: Val Loss: 2.7963, Val F1macro: 0.0693,Test Accuracy: 0.125, Test F1-macro: 0.069
01/002: Val Loss: 2.7593, Val F1macro: 0.0644,Test Accuracy: 0.103, Test F1-macro: 0.060
01/003: Val Loss: 2.8024, Val F1macro: 0.0893,Test Accuracy: 0.113, Test F1-macro: 0.100
01/004: Val Loss: 3.1248, Val F1macro: 0.0333,Test Accuracy: 0.084, Test F1-macro: 0.036
01/005: Val Loss: 2.7358, Val F1macro: 0.1189,Test Accuracy: 0.128, Test F1-macro: 0.124
01/006: Val Loss: 2.8043, Val F1macro: 0.0667,Test Accuracy: 0.098, Test F1-macro: 0.058
01/007: Val Loss: 2.7249, Val F1macro: 0.0633,Test Accuracy: 0.123, Test F1-macro: 0.057
01/008: Val Loss: 2.8596, Val F1macro: 0.0630,Test Accuracy: 0.112, Test F1-macro: 0.054
01/009: Val Loss: 2.9712, Val F1macro: 0.0462,Test Accuracy: 0.096, Test F1-macro: 0.041
01/010: 

01/091: Val Loss: 2.6816, Val F1macro: 0.1838,Test Accuracy: 0.174, Test F1-macro: 0.190
01/092: Val Loss: 2.6531, Val F1macro: 0.1843,Test Accuracy: 0.162, Test F1-macro: 0.176
01/093: Val Loss: 2.6461, Val F1macro: 0.1840,Test Accuracy: 0.166, Test F1-macro: 0.180
01/094: Val Loss: 2.6427, Val F1macro: 0.1578,Test Accuracy: 0.160, Test F1-macro: 0.162
01/095: Val Loss: 2.6415, Val F1macro: 0.1796,Test Accuracy: 0.177, Test F1-macro: 0.192
01/096: Val Loss: 2.6576, Val F1macro: 0.1877,Test Accuracy: 0.171, Test F1-macro: 0.191
01/097: Val Loss: 2.6531, Val F1macro: 0.1609,Test Accuracy: 0.163, Test F1-macro: 0.157
01/098: Val Loss: 2.6524, Val F1macro: 0.1816,Test Accuracy: 0.172, Test F1-macro: 0.188
01/099: Val Loss: 2.6578, Val F1macro: 0.1770,Test Accuracy: 0.169, Test F1-macro: 0.172
01/100: Val Loss: 2.6404, Val F1macro: 0.1793,Test Accuracy: 0.173, Test F1-macro: 0.186
01/101: Val Loss: 2.6439, Val F1macro: 0.1808,Test Accuracy: 0.175, Test F1-macro: 0.188
01/102: Val Loss: 2.6

01/184: Val Loss: 2.7344, Val F1macro: 0.1898,Test Accuracy: 0.179, Test F1-macro: 0.198
01/185: Val Loss: 2.6361, Val F1macro: 0.1891,Test Accuracy: 0.179, Test F1-macro: 0.197
01/186: Val Loss: 2.6442, Val F1macro: 0.1893,Test Accuracy: 0.182, Test F1-macro: 0.200
01/187: Val Loss: 2.6360, Val F1macro: 0.1769,Test Accuracy: 0.171, Test F1-macro: 0.178
01/188: Val Loss: 2.6306, Val F1macro: 0.1920,Test Accuracy: 0.176, Test F1-macro: 0.197
01/189: Val Loss: 2.6510, Val F1macro: 0.1844,Test Accuracy: 0.175, Test F1-macro: 0.192
01/190: Val Loss: 2.6336, Val F1macro: 0.1803,Test Accuracy: 0.176, Test F1-macro: 0.187
01/191: Val Loss: 2.6341, Val F1macro: 0.1899,Test Accuracy: 0.178, Test F1-macro: 0.199
01/192: Val Loss: 2.6300, Val F1macro: 0.1906,Test Accuracy: 0.178, Test F1-macro: 0.199
01/193: Val Loss: 2.6712, Val F1macro: 0.1881,Test Accuracy: 0.179, Test F1-macro: 0.196
01/194: Val Loss: 2.6342, Val F1macro: 0.1888,Test Accuracy: 0.178, Test F1-macro: 0.196
01/195: Val Loss: 2.6

01/277: Val Loss: 2.6230, Val F1macro: 0.1880,Test Accuracy: 0.181, Test F1-macro: 0.202
01/278: Val Loss: 2.6272, Val F1macro: 0.1885,Test Accuracy: 0.180, Test F1-macro: 0.202
01/279: Val Loss: 2.6207, Val F1macro: 0.1886,Test Accuracy: 0.182, Test F1-macro: 0.199
01/280: Val Loss: 2.6205, Val F1macro: 0.1875,Test Accuracy: 0.183, Test F1-macro: 0.201
01/281: Val Loss: 2.6276, Val F1macro: 0.1949,Test Accuracy: 0.182, Test F1-macro: 0.201
01/282: Val Loss: 2.6333, Val F1macro: 0.1918,Test Accuracy: 0.179, Test F1-macro: 0.198
01/283: Val Loss: 2.6528, Val F1macro: 0.1919,Test Accuracy: 0.184, Test F1-macro: 0.203
01/284: Val Loss: 2.6467, Val F1macro: 0.1855,Test Accuracy: 0.172, Test F1-macro: 0.179
01/285: Val Loss: 2.6859, Val F1macro: 0.1954,Test Accuracy: 0.174, Test F1-macro: 0.200
01/286: Val Loss: 2.6335, Val F1macro: 0.1962,Test Accuracy: 0.178, Test F1-macro: 0.202
01/287: Val Loss: 2.6272, Val F1macro: 0.1894,Test Accuracy: 0.177, Test F1-macro: 0.194
01/288: Val Loss: 2.6

01/370: Val Loss: 2.6268, Val F1macro: 0.1875,Test Accuracy: 0.179, Test F1-macro: 0.197
01/371: Val Loss: 2.6833, Val F1macro: 0.1835,Test Accuracy: 0.163, Test F1-macro: 0.180
01/372: Val Loss: 2.6413, Val F1macro: 0.1880,Test Accuracy: 0.178, Test F1-macro: 0.197
01/373: Val Loss: 2.6633, Val F1macro: 0.1750,Test Accuracy: 0.165, Test F1-macro: 0.184
01/374: Val Loss: 2.6345, Val F1macro: 0.1934,Test Accuracy: 0.178, Test F1-macro: 0.202
01/375: Val Loss: 2.6386, Val F1macro: 0.1925,Test Accuracy: 0.172, Test F1-macro: 0.190
01/376: Val Loss: 2.6406, Val F1macro: 0.1870,Test Accuracy: 0.177, Test F1-macro: 0.195
01/377: Val Loss: 2.7640, Val F1macro: 0.1852,Test Accuracy: 0.168, Test F1-macro: 0.187
01/378: Val Loss: 2.6501, Val F1macro: 0.1862,Test Accuracy: 0.173, Test F1-macro: 0.192
01/379: Val Loss: 2.6266, Val F1macro: 0.1876,Test Accuracy: 0.182, Test F1-macro: 0.199
01/380: Val Loss: 2.7267, Val F1macro: 0.1921,Test Accuracy: 0.179, Test F1-macro: 0.198
01/381: Val Loss: 2.6

...

03/114: Val Loss: 2.6939, Val F1macro: 0.1594,Test Accuracy: 0.153, Test F1-macro: 0.150
03/115: Val Loss: 2.7573, Val F1macro: 0.1076,Test Accuracy: 0.130, Test F1-macro: 0.106
03/116: Val Loss: 2.6542, Val F1macro: 0.1607,Test Accuracy: 0.151, Test F1-macro: 0.147
03/117: Val Loss: 2.7714, Val F1macro: 0.1620,Test Accuracy: 0.151, Test F1-macro: 0.157
03/118: Val Loss: 2.7273, Val F1macro: 0.1761,Test Accuracy: 0.165, Test F1-macro: 0.169
03/119: Val Loss: 2.8126, Val F1macro: 0.1019,Test Accuracy: 0.118, Test F1-macro: 0.102
03/120: Val Loss: 2.7848, Val F1macro: 0.1584,Test Accuracy: 0.152, Test F1-macro: 0.149
03/121: Val Loss: 2.6621, Val F1macro: 0.1608,Test Accuracy: 0.154, Test F1-macro: 0.150
03/122: Val Loss: 2.6513, Val F1macro: 0.1796,Test Accuracy: 0.156, Test F1-macro: 0.166
03/123: Val Loss: 2.7247, Val F1macro: 0.1573,Test Accuracy: 0.153, Test F1-macro: 0.147
03/124: Val Loss: 2.8599, Val F1macro: 0.1100,Test Accuracy: 0.122, Test F1-macro: 0.105
03/125: Val Loss: 2.7

03/207: Val Loss: 2.9797, Val F1macro: 0.1753,Test Accuracy: 0.171, Test F1-macro: 0.172
03/208: Val Loss: 2.6544, Val F1macro: 0.1776,Test Accuracy: 0.174, Test F1-macro: 0.172
03/209: Val Loss: 2.7938, Val F1macro: 0.1791,Test Accuracy: 0.177, Test F1-macro: 0.174
03/210: Val Loss: 2.6751, Val F1macro: 0.1759,Test Accuracy: 0.173, Test F1-macro: 0.174
03/211: Val Loss: 2.6773, Val F1macro: 0.1647,Test Accuracy: 0.168, Test F1-macro: 0.162
03/212: Val Loss: 2.6389, Val F1macro: 0.1780,Test Accuracy: 0.179, Test F1-macro: 0.175
03/213: Val Loss: 2.6759, Val F1macro: 0.1778,Test Accuracy: 0.179, Test F1-macro: 0.176
03/214: Val Loss: 2.6790, Val F1macro: 0.1780,Test Accuracy: 0.179, Test F1-macro: 0.177
03/215: Val Loss: 2.6477, Val F1macro: 0.1794,Test Accuracy: 0.181, Test F1-macro: 0.180
03/216: Val Loss: 2.6798, Val F1macro: 0.1764,Test Accuracy: 0.179, Test F1-macro: 0.174
03/217: Val Loss: 2.9533, Val F1macro: 0.1779,Test Accuracy: 0.176, Test F1-macro: 0.176
03/218: Val Loss: 2.6

03/300: Val Loss: 3.2974, Val F1macro: 0.1803,Test Accuracy: 0.173, Test F1-macro: 0.179
03/301: Val Loss: 2.7761, Val F1macro: 0.1815,Test Accuracy: 0.175, Test F1-macro: 0.172
03/302: Val Loss: 2.7916, Val F1macro: 0.1768,Test Accuracy: 0.175, Test F1-macro: 0.172
03/303: Val Loss: 2.7433, Val F1macro: 0.1811,Test Accuracy: 0.179, Test F1-macro: 0.174
03/304: Val Loss: 2.7363, Val F1macro: 0.1793,Test Accuracy: 0.179, Test F1-macro: 0.174
03/305: Val Loss: 2.6898, Val F1macro: 0.1808,Test Accuracy: 0.177, Test F1-macro: 0.172
03/306: Val Loss: 2.7314, Val F1macro: 0.1827,Test Accuracy: 0.174, Test F1-macro: 0.178
03/307: Val Loss: 2.8502, Val F1macro: 0.1440,Test Accuracy: 0.125, Test F1-macro: 0.139
03/308: Val Loss: 2.7258, Val F1macro: 0.1855,Test Accuracy: 0.177, Test F1-macro: 0.177
03/309: Val Loss: 2.8613, Val F1macro: 0.1779,Test Accuracy: 0.177, Test F1-macro: 0.173
03/310: Val Loss: 2.7397, Val F1macro: 0.1608,Test Accuracy: 0.158, Test F1-macro: 0.162
03/311: Val Loss: 2.6

03/393: Val Loss: 2.6677, Val F1macro: 0.1770,Test Accuracy: 0.179, Test F1-macro: 0.173
03/394: Val Loss: 2.9369, Val F1macro: 0.1634,Test Accuracy: 0.160, Test F1-macro: 0.162
03/395: Val Loss: 2.6246, Val F1macro: 0.1797,Test Accuracy: 0.177, Test F1-macro: 0.172
03/396: Val Loss: 2.6454, Val F1macro: 0.1731,Test Accuracy: 0.167, Test F1-macro: 0.162
03/397: Val Loss: 2.6603, Val F1macro: 0.1687,Test Accuracy: 0.174, Test F1-macro: 0.164
03/398: Val Loss: 2.6794, Val F1macro: 0.1633,Test Accuracy: 0.165, Test F1-macro: 0.161
03/399: Val Loss: 2.6432, Val F1macro: 0.1697,Test Accuracy: 0.171, Test F1-macro: 0.161
03/400: Val Loss: 2.7815, Val F1macro: 0.1640,Test Accuracy: 0.161, Test F1-macro: 0.165
03/401: Val Loss: 2.7497, Val F1macro: 0.1638,Test Accuracy: 0.162, Test F1-macro: 0.158
03/402: Val Loss: 2.7186, Val F1macro: 0.1675,Test Accuracy: 0.172, Test F1-macro: 0.161
03/403: Val Loss: 2.8329, Val F1macro: 0.1662,Test Accuracy: 0.159, Test F1-macro: 0.166
03/404: Val Loss: 2.6

03/486: Val Loss: 2.7938, Val F1macro: 0.1723,Test Accuracy: 0.176, Test F1-macro: 0.167
03/487: Val Loss: 2.7331, Val F1macro: 0.1769,Test Accuracy: 0.174, Test F1-macro: 0.170
03/488: Val Loss: 2.6669, Val F1macro: 0.1780,Test Accuracy: 0.180, Test F1-macro: 0.173
03/489: Val Loss: 2.7224, Val F1macro: 0.1784,Test Accuracy: 0.179, Test F1-macro: 0.172
03/490: Val Loss: 2.8491, Val F1macro: 0.1720,Test Accuracy: 0.177, Test F1-macro: 0.168
03/491: Val Loss: 2.8740, Val F1macro: 0.1627,Test Accuracy: 0.162, Test F1-macro: 0.161
03/492: Val Loss: 2.6568, Val F1macro: 0.1648,Test Accuracy: 0.169, Test F1-macro: 0.162
03/493: Val Loss: 2.7469, Val F1macro: 0.1661,Test Accuracy: 0.165, Test F1-macro: 0.171
03/494: Val Loss: 2.7789, Val F1macro: 0.1775,Test Accuracy: 0.178, Test F1-macro: 0.171
03/495: Val Loss: 2.7130, Val F1macro: 0.1812,Test Accuracy: 0.177, Test F1-macro: 0.174
03/496: Val Loss: 2.9719, Val F1macro: 0.1635,Test Accuracy: 0.160, Test F1-macro: 0.161
03/497: Val Loss: 2.7